In [1]:
import pickle

import numpy as np
import pandas as pd 

from importlib import reload

import helper_funcs;reload(helper_funcs);from helper_funcs import *
import make_models2;reload(make_models2);from make_models2 import *

from functools import partial
from sklearn.decomposition import PCA

from matplotlib import pyplot as plt

from datetime import datetime

import shutil
from functools import partial
from tqdm import tqdm
tqdm = partial(tqdm, position=0, leave=True)

In [2]:
# GPU checks
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [3]:
!nvidia-smi

Tue Oct  4 10:06:21 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:18:00.0 Off |                    0 |
| N/A   31C    P0    57W / 300W |    484MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:3B:00.0 Off |                    0 |
| N/A   

In [4]:
ABIDE_data = np.load(os.path.expanduser('~/Documents/Bilingualism_CVAE/data/array_brains/arr_combine/ABIDE_data.npy'))

In [5]:
latent_dim=32;batch_size=32;disentangle=False;gamma=100
encoder, decoder, vae = get_MRI_VAE_3D(input_shape=(64, 64, 64, 1), latent_dim=latent_dim, batch_size=batch_size, disentangle=disentangle, gamma=gamma)

encoder, decoder, vae = get_MRI_VAE_3D(input_shape=(64,64,64,1), 
                                            latent_dim=32, 
                                            batch_size = batch_size, 
                                            disentangle=True,
                                            gamma=100,
                                            kernel_size = 3,
                                            filters = 48,
                                            intermediate_dim = 128,
                                            nlayers = 2,
                                            bias=True)

fn = os.path.expanduser('~/Documents/Bilingualism_CVAE/data/tf_outputs/VAE/VAE_weights')
vae.load_weights(fn)

In [6]:
# LOAD CVAE
latent_dim = 16;batch_size = 32;beta = 1;gamma = 100;disentangle = True
cvae, z_encoder, s_encoder, cvae_decoder = get_MRI_CVAE_3D(latent_dim=latent_dim,beta=beta, disentangle=disentangle, gamma=gamma, bias=True, batch_size = batch_size)
loss = list()    

fn = os.path.expanduser('~/Documents/Bilingualism_CVAE/data/tf_outputs/CVAE/CVAE_weights')
cvae.load_weights(fn)

In [7]:
n_samples = 10

salient_vec_abide = np.array([s_encoder.predict(ABIDE_data[:,:,:,:])[2] for _ in range(n_samples)])
background_vec_abide = np.array([z_encoder.predict(ABIDE_data[:,:,:,:])[2] for _ in range(n_samples)])
vae_vec_abide = np.array([encoder.predict(ABIDE_data[:,:,:,:])[2] for _ in range(n_samples)])

# salient_vec_sfari = np.array([s_encoder.predict(SFARI_data[:,:,:,:])[2] for _ in range(n_samples)])
# background_vec_sfari = np.array([z_encoder.predict(SFARI_data[:,:,:,:])[2] for _ in range(n_samples)])
# vae_vec_sfari = np.array([encoder.predict(SFARI_data[:,:,:,:])[2] for _ in range(n_samples)])

fn = os.path.expanduser('~/Documents/Bilingualism_CVAE/data/latent_vecs10.npz')
np.savez_compressed(fn, 
                    salient_vec_abide=salient_vec_abide,
                    background_vec_abide=background_vec_abide,
                     vae_vec_abide=vae_vec_abide)
#                     salient_vec_sfari=salient_vec_sfari,
#                     background_vec_sfari=background_vec_sfari,
#                     vae_vec_sfari=vae_vec_sfari)

5/5 [==============================] - 0s 23ms/step
